# using sqlparse to create meta data
limitations
- when using spaces in tables names with reserved keyworkds, e.g. case in "tableau enc case view"

Have solved this by changing sq_metadata

In [1]:
!conda info --envs

# conda environments:
#
base                     /home/davidgriffiths/miniconda3
sqlparse              *  /home/davidgriffiths/miniconda3/envs/sqlparse



In [2]:
import sql_metadata

In [17]:
import re
from typing import List, Tuple, Optional, Dict, Symbol
import sqlparse

from sqlparse.sql import TokenList
from sqlparse.tokens import Name, Whitespace, Wildcard, Number, Punctuation

# simple example

In [3]:
sql_metadata.get_query_tokens("SELECT * FROM foo")

[<DML 'SELECT' at 0x7FC7AC0C9D68>,
 <Wildcard '*' at 0x7FC7AC0C9BE8>,
 <Keyword 'FROM' at 0x7FC7AC0C9B28>,
 <Name 'foo' at 0x7FC7AC0C9DC8>]

In [4]:
sql_metadata.get_query_columns("SELECT test, id FROM foo, bar")

['test', 'id']

In [5]:
sql_metadata.get_query_tables("SELECT a.* FROM product_a.users AS a JOIN product_b.users AS b ON a.ip_address = b.ip_address")

['product_a.users', 'product_b.users']

# example query

In [9]:
query='''
    select "Case Snap Date", count(*)
FROM "enc_dde"."tableau enc case field links view" "A61S Data Quality Indicators"
  INNER JOIN "enc_dde"."tableau enc case view" "A61S Case" ON ("A61S Data Quality Indicators"."case_key" = "A61S Case"."case_key")
  INNER JOIN "enc_dde"."tableau enc case snapshot view" "A61S Case Snapshot" ON ("A61S Data Quality Indicators"."case_key" = "A61S Case Snapshot"."case_key")
  INNER JOIN "nbn_dde_23"."tableau location view" "A61S Location" ON ("A61S Data Quality Indicators"."location_key" = "A61S Location"."location key")
  where "Case Snap Date" > current_date-4
  group by 1'''

In [10]:
sql_metadata.get_query_tables(query)

['enc_dde.tableau', 'nbn_dde_23.tableau']

# re built get_query_tables
i remvoed the quote removal

In [74]:
get_query_tables(query)

['"enc_dde"."tableau enc case field links view"',
 '"enc_dde"."tableau enc case view"',
 '"enc_dde"."tableau enc case snapshot view"',
 '"nbn_dde_23"."tableau location view"']

In [73]:
def get_query_tables(query: str) -> List[str]:
    """
    :type query str
    :rtype: list[str]
    """
    tables = []
    last_keyword = None

    table_syntax_keywords = [
        # SELECT queries
        'FROM', 'WHERE', 'JOIN', 'INNER JOIN', 'FULL JOIN', 'FULL OUTER JOIN',
        'LEFT OUTER JOIN', 'RIGHT OUTER JOIN',
        'LEFT JOIN', 'RIGHT JOIN', 'ON',
        # INSERT queries
        'INTO', 'VALUES',
        # UPDATE queries
        'UPDATE', 'SET',
        # Hive queries
        'TABLE',  # INSERT TABLE
    ]

    # print(query, get_query_tokens(query))
#     query = query.replace('"', '')
    tokens = sql_metadata.get_query_tokens(query)

    for index, token in enumerate(tokens):
#         print([token, token.ttype, last_keyword])
        if token.is_keyword and token.value.upper() in table_syntax_keywords:
            # keep the name of the last keyword, the next one can be a table name
            last_keyword = token.value.upper()
            # print('keyword', last_keyword)
        elif str(token) == '(':
            # reset the last_keyword for INSERT `foo` VALUES(id, bar) ...
            last_keyword = None
        elif token.is_keyword and str(token) in ['FORCE', 'ORDER', 'GROUP BY']:
            # reset the last_keyword for queries like:
            # "SELECT x FORCE INDEX"
            # "SELECT x ORDER BY"
            # "SELECT x FROM y GROUP BY x"
            last_keyword = None
        elif token.is_keyword and str(token) == 'SELECT' and last_keyword in ['INTO', 'TABLE']:
            # reset the last_keyword for "INSERT INTO SELECT" and "INSERT TABLE SELECT" queries
            last_keyword = None
        elif token.ttype is Name or token.ttype or token.is_keyword:
            tables = sql_metadata._update_table_names(tables, tokens, index, last_keyword)

    return sql_metadata.unique(tables)